In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
import json

In [12]:
combined = pd.read_csv('/Users/pranmodu/Library/Mobile Documents/com~apple~CloudDocs/Desktop/coding projects/apart-sprint/data/processed/combined_benchmarks.csv')

# Show initial statistics
print(f"Initial loaded data:")
print(f"Total rows: {len(combined)}")
print(f"Rows with missing model: {combined['model'].isna().sum()}")
print(f"Rows with missing org: {combined['org'].isna().sum()}")

combined

Initial loaded data:
Total rows: 2827
Rows with missing model: 401
Rows with missing org: 595


,model,benchmark,date,org,country,training_compute_flops,score,capability
0,gpt-5-2025-08-07_high,aider_polyglot,2025-08-07,OpenAI,United States of America,6.600000e+25,88.000,code_generation
1,gpt-5-2025-08-07_medium,aider_polyglot,2025-08-07,OpenAI,United States of America,6.600000e+25,86.700,code_generation
2,o3-pro-2025-06-10_high,aider_polyglot,2025-06-10,OpenAI,United States of America,NaN,84.900,code_generation
3,gemini-2.5-pro-preview-06-05_32K,aider_polyglot,NaN,Google DeepMind,"United States of America,United Kingdom of Gre...",NaN,83.100,code_generation
4,gpt-5-2025-08-07_low,aider_polyglot,2025-08-07,OpenAI,United States of America,6.600000e+25,81.300,code_generation
...,...,...,...,...,...,...,...,...
2822,opt-13b,hella_swag,2022-05-11,NaN,NaN,NaN,0.699,commonsense_reasoning
2823,gpt-j-6b,hella_swag,2021-08-05,"EleutherAI,LAION","United States of America,Germany",1.500000e+22,0.662,commonsense_reasoning
2824,dolly-v2-12b,hella_swag,2023-04-11,Databricks,United States of America,NaN,0.708,commonsense_reasoning
2825,Cerebras-GPT-13B,hella_swag,2023-03-20,Cerebras Systems,United States of America,2.300000e+22,0.594,commonsense_reasoning


In [13]:
combined['date'] = pd.to_datetime(combined['date'])

# Create year column as integer (Int64 to handle NaN values properly)
combined['year'] = combined['date'].dt.year.astype('Int64')

# First, remove rows with empty model column
print(f"Step 1: Removing rows with empty model column")
print(f"  Before: {len(combined)} rows, {combined['org'].isna().sum()} missing orgs")
combined = combined.dropna(subset=['model'])
print(f"  After: {len(combined)} rows, {combined['org'].isna().sum()} missing orgs")
print(f"  Removed: {2827 - len(combined)} rows")

# Now fill missing org values for models that have org in other rows
print(f"\nStep 2: Filling missing org values from duplicate models")
models_missing_org = combined[combined['org'].isna()]['model'].unique()
print(f"  Unique models with missing org: {len(models_missing_org)}")

filled_count = 0
# For each model with missing org, try to find org from other rows with the same model
for model in models_missing_org:
    # Get all rows with this model that have an org
    model_with_org = combined[(combined['model'] == model) & (combined['org'].notna())]
    
    if len(model_with_org) > 0:
        # Get the most common org for this model
        org_value = model_with_org['org'].mode()[0] if len(model_with_org['org'].mode()) > 0 else model_with_org['org'].iloc[0]
        
        # Count how many will be filled
        rows_to_fill = ((combined['model'] == model) & (combined['org'].isna())).sum()
        filled_count += rows_to_fill
        
        # Fill missing org for this model
        combined.loc[(combined['model'] == model) & (combined['org'].isna()), 'org'] = org_value
        print(f"  Filled org '{org_value}' for {rows_to_fill} row(s) of model '{model}'")

# Check final stats
final_missing = combined['org'].isna().sum()
print(f"\nFinal results:")
print(f"  Total rows: {len(combined)}")
print(f"  Filled orgs: {filled_count}")
print(f"  Still missing org: {final_missing}")
print(f"  (These {final_missing} models don't have org info anywhere in the dataset)")

Step 1: Removing rows with empty model column
  Before: 2827 rows, 595 missing orgs
  After: 2426 rows, 194 missing orgs
  Removed: 401 rows

Step 2: Filling missing org values from duplicate models
  Unique models with missing org: 61

Final results:
  Total rows: 2426
  Filled orgs: 0
  Still missing org: 194
  (These 194 models don't have org info anywhere in the dataset)


In [16]:
# Apply recommended cleaning steps
print("=" * 60)
print("CLEANING DATA")
print("=" * 60)

print(f"\nStarting with: {len(combined)} rows")

# Step 1: Remove rows with missing score
before = len(combined)
combined = combined.dropna(subset=['score'])
removed = before - len(combined)
print(f"\n1. Removed {removed} rows with missing score")
print(f"   Remaining: {len(combined)} rows")

# Step 2: Remove rows with missing date/year
before = len(combined)
combined = combined.dropna(subset=['date'])
removed = before - len(combined)
print(f"\n2. Removed {removed} rows with missing date")
print(f"   Remaining: {len(combined)} rows")

# Step 3: Remove exact duplicate rows
before = len(combined)
combined = combined.drop_duplicates()
removed = before - len(combined)
print(f"\n3. Removed {removed} exact duplicate rows")
print(f"   Remaining: {len(combined)} rows")

# Summary
print("\n" + "=" * 60)
print(f"FINAL CLEANED DATA: {len(combined)} rows")
print("=" * 60)
print(f"Missing values:")
print(f"  model: {combined['model'].isna().sum()}")
print(f"  score: {combined['score'].isna().sum()}")
print(f"  date: {combined['date'].isna().sum()}")
print(f"  year: {combined['year'].isna().sum()}")
print(f"  org: {combined['org'].isna().sum()}")

# Save to CSV
output_path = '/Users/pranmodu/Library/Mobile Documents/com~apple~CloudDocs/Desktop/coding projects/apart-sprint/data/processed/combined_benchmarks_cleaned.csv'
combined.to_csv(output_path, index=False)
print(f"\nSaved to: {output_path}")

CLEANING DATA

Starting with: 2426 rows

1. Removed 34 rows with missing score
   Remaining: 2392 rows

2. Removed 62 rows with missing date
   Remaining: 2330 rows

3. Removed 245 exact duplicate rows
   Remaining: 2085 rows

FINAL CLEANED DATA: 2085 rows
Missing values:
  model: 0
  score: 0
  date: 0
  year: 0
  org: 166

Saved to: /Users/pranmodu/Library/Mobile Documents/com~apple~CloudDocs/Desktop/coding projects/apart-sprint/data/processed/combined_benchmarks_cleaned.csv


In [20]:
combined['org'].value_counts()

org
OpenAI                                                                                                                                                                 380
Meta AI                                                                                                                                                                338
Anthropic                                                                                                                                                              234
Google DeepMind                                                                                                                                                        146
Alibaba                                                                                                                                                                118
Mistral AI                                                                                                                                   